In [ ]:
!pip install geemap -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import ee
import geemap
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
import folium
from folium import TileLayer
import random
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import datetime
import time

In [ ]:
# Authenticate to the Earth Engine servers
ee.Authenticate()
# Initialize the Earth Engine object with Google Cloud project ID
project_id = ('ee-janetmumo68') # you can change to your id
ee.Initialize(project=project_id)

# Exploring the data


*   Sizes of the data: 2018- 3598,  2019 - 1434, 2020 - 7022, 2023-4322
*  The geometry types in the 2019, 2020 data (Polygon, MultiPolygon, and GeometryCollection-multiple geometry types)
*   Does the GeometryCollection type mean overlapping plots?
*   The extracted data is exported as GeoJSON which supports mixed geometry types
*   2020 data has some specutio/types that need to be changed from crop types to no crop type such as  soil, pavel roads, built-up areas etc




In [ ]:
# Function to load individual crop feature collections
def load_crop_data(asset_id):
    return ee.FeatureCollection(asset_id)

# Load datasets from 2018-2023
data_2018 = ee.FeatureCollection("projects/ee-djitastar/assets/data_2018")
data_2019 = ee.FeatureCollection("projects/ee-djitastar/assets/data_2019")
data_2020 = ee.FeatureCollection("projects/ee-djitastar/assets/data_2020")
# Load the Senegal Admin1 boundaries
senegal_admin1 = ee.FeatureCollection("projects/ee-janet/assets/senegal/sen_adm1")

# Load individual rainfed 2023 crop feature collections
cashew = load_crop_data('projects/ee-kkidia3/assets/cashew') #tree_crop
cassava = load_crop_data('projects/ee-kkidia3/assets/cassava') #vegitable
cowpea = load_crop_data('projects/ee-kkidia3/assets/cowpea') #legumne
cowpmix = load_crop_data('projects/ee-kkidia3/assets/cowpea_mixed') #mix_legu
eggplant = load_crop_data('projects/ee-kkidia3/assets/eggplant') #vegitable
fallow = load_crop_data('projects/ee-kkidia3/assets/fallow') #fallow
fonio = load_crop_data('projects/ee-kkidia3/assets/fonio') #Cereals
gnut = load_crop_data('projects/ee-kkidia3/assets/groundnut') #legume
gnutmix = load_crop_data('projects/ee-kkidia3/assets/groundnut_mixed') #mix_legu
gsorrel = load_crop_data('projects/ee-kkidia3/assets/guinea_sorrel') #vegetable
maize = load_crop_data('projects/ee-kkidia3/assets/maize') #Cereal
millet = load_crop_data('projects/ee-kkidia3/assets/millet') #cereal
milletmix = load_crop_data('projects/ee-kkidia3/assets/millet_mixed') #mix_cereal
okra = load_crop_data('projects/ee-kkidia3/assets/okra') #vegitable
potato = load_crop_data('projects/ee-kkidia3/assets/potato') #vegitable
rice = load_crop_data('projects/ee-kkidia3/assets/rice') #cereal
sesame = load_crop_data('projects/ee-kkidia3/assets/sesame') #ceareal?
sorgh = load_crop_data('projects/ee-kkidia3/assets/sorghum') #cereal
soye = load_crop_data('projects/ee-kkidia3/assets/soye') #cereal?
squash = load_crop_data('projects/ee-kkidia3/assets/squash') #vegitable
taro = load_crop_data('projects/ee-kkidia3/assets/taro') #vegitable
vouandzou = load_crop_data('projects/ee-kkidia3/assets/vouandzou') #
melon = load_crop_data('projects/ee-kkidia3/assets/watermelon') #vegitable
wheat = load_crop_data('projects/ee-kkidia3/assets/wheat') #cereal

# Merge all crop fields into a single feature collection except fallow as not part of crop field
merged2023 = gnut.merge(gsorrel).merge(cashew).merge(cassava).merge(cowpea)\
                .merge(eggplant).merge(fonio).merge(gsorrel)\
                .merge(maize).merge(millet).merge(okra).merge(potato)\
                .merge(rice).merge(sesame).merge(sorgh).merge(soye).merge(squash).merge(taro)\
                .merge(vouandzou).merge(melon).merge(wheat).merge(milletmix).merge(gnutmix).merge(cowpmix)#.merge(fallow)

fallow = fallow.filter(ee.Filter.eq('class', 'Fallow')) #.merge(fallow) is not a crop

Investigate the geometry types

In [ ]:
def check_geometry_type(feature):
    return feature.set('geom_type', feature.geometry().type())

data_2019_with_type = data_2019.map(check_geometry_type)

# Get unique geometry types
unique_types = data_2019_with_type.aggregate_array('geom_type').distinct()
print("Unique geometry types:", unique_types.getInfo())

# Count features of each type
for geom_type in unique_types.getInfo():
    count = data_2019_with_type.filter(ee.Filter.eq('geom_type', geom_type)).size().getInfo()
    print(f"Number of features with {geom_type} geometry: {count}")

Unique geometry types: ['Polygon', 'GeometryCollection', 'MultiPolygon']
Number of features with Polygon geometry: 1429
Number of features with GeometryCollection geometry: 1
Number of features with MultiPolygon geometry: 4


In [ ]:
def check_geometry_type(feature):
    return feature.set('geom_type', feature.geometry().type())

data_2020_with_type = data_2020.map(check_geometry_type)

# Get unique geometry types
unique_types = data_2020_with_type.aggregate_array('geom_type').distinct()
print("Unique geometry types:", unique_types.getInfo())

# Count features of each type
for geom_type in unique_types.getInfo():
    count = data_2020_with_type.filter(ee.Filter.eq('geom_type', geom_type)).size().getInfo()
    print(f"Number of features with {geom_type} geometry: {count}")

Unique geometry types: ['Polygon', 'MultiPolygon', 'GeometryCollection']
Number of features with Polygon geometry: 6235
Number of features with MultiPolygon geometry: 743
Number of features with GeometryCollection geometry: 44


Print the data as dataframe

In [ ]:
# Get the first few features to display their properties
features_to_display = data_2020.limit(5000).getInfo()['features']
# Extract properties into a list of dictionaries
properties_list = [feature['properties'] for feature in features_to_display]
# Create a DataFrame from the properties list
df = pd.DataFrame(properties_list)

df.head() #show it in data fram

# Potential remote sensing indices for classification
These indices will be calculated and exported the user can choose which ones to use as fetaures
1. NDVI (Normalized Difference Vegetation Index):
   - Formula: (NIR - RED) / (NIR + RED)
   - Measures vegetation health and density

2. EVI (Enhanced Vegetation Index):
   - Formula: 2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))
   - Improved sensitivity in high biomass regions and better soil background correction

3. SAVI (Soil Adjusted Vegetation Index):
   - Formula: ((NIR - RED) / (NIR + RED + 0.5)) * 1.5
   - Minimizes soil brightness influences in areas with partial vegetation cover

4. NDWI (Normalized Difference Water Index):
   - Formula: (NIR - SWIR) / (NIR + SWIR)
   - Measures vegetation water content and water bodies

5. NDBI (Normalized Difference Built-up Index):
   - Formula: (SWIR - NIR) / (SWIR + NIR)
   - Highlights urban areas and built-up land

6. NBR (Normalized Burn Ratio):
   - Formula: (NIR - SWIR) / (NIR + SWIR)
   - Identifies burned areas and estimates burn severity

7. GCI (Green Chlorophyll Index):
   - Formula: (NIR / GREEN) - 1
   - Estimates chlorophyll content in leaves

8. NDRE (Normalized Difference Red Edge):
   - Formula: (NIR - RED EDGE) / (NIR + RED EDGE)
   - Sensitive to chlorophyll content, useful for crop monitoring

#### a. Calculate the indices and export them as Vector data file including the normalized bands values
- Class Definition: Sentinel2Processor class encapsulates the functionality for processing Sentinel-2 imagery.
- Initialization: The class is initialized with the year, months, indices to calculate, and the feature collection to process.
- The calculate_indices_and_normalize method calculates the specified indices and normalizes the bands
- Monthly Processing (time series for 3 months): The process_month method processes imagery for a specific month and renames the bands with a date suffix.
- Exporting: The export_to_drive method handles exporting the processed data to Google Drive, with a polling mechanism to monitor the task status.

In [ ]:
class Sentinel2Processor:
    def __init__(self, year, months, indices, data_fc):
        self.year = year
        self.months = months
        self.indices = indices
        self.data_fc = data_fc

    def calculate_indices_and_normalize(self, image):
        index_functions = {
            'NDVI': lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI'),
            'EVI': lambda img: img.expression(
                '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
                {'NIR': img.select('B8'), 'RED': img.select('B4'), 'BLUE': img.select('B2')}
            ).rename('EVI'),
            'SAVI': lambda img: img.expression(
                '((NIR - RED) / (NIR + RED + 0.5)) * (1.5)',
                {'NIR': img.select('B8'), 'RED': img.select('B4')}
            ).rename('SAVI'),
            'NDWI': lambda img: img.normalizedDifference(['B8', 'B11']).rename('NDWI'),
            'NDBI': lambda img: img.normalizedDifference(['B11', 'B8']).rename('NDBI'),
            'NBR': lambda img: img.normalizedDifference(['B8', 'B12']).rename('NBR'),
            'GCI': lambda img: img.expression(
                '(NIR / GREEN) - 1',
                {'NIR': img.select('B8'), 'GREEN': img.select('B3')}
            ).rename('GCI'),
            'NDRE': lambda img: img.normalizedDifference(['B8', 'B5']).rename('NDRE')
        }

        # Calculate specified indices
        bands = []
        for index in self.indices:
            if index in index_functions:
                bands.append(index_functions[index](image))

        # Add indices to the image
        image_with_indices = image.addBands(bands)

        # Define bands to normalize
        bands_to_normalize = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

        # Normalize bands
        def normalize_band(band):
            min_val = image_with_indices.select(band).reduceRegion(
                reducer=ee.Reducer.min(),
                geometry=image_with_indices.geometry(),
                scale=10,
                maxPixels=1e13
            ).get(band)
            max_val = image_with_indices.select(band).reduceRegion(
                reducer=ee.Reducer.max(),
                geometry=image_with_indices.geometry(),
                scale=10,
                maxPixels=1e13
            ).get(band)
            normalized = image_with_indices.select(band).subtract(ee.Number(min_val)).divide(
                ee.Number(max_val).subtract(ee.Number(min_val))
            ).rename(band + '_n')
            return normalized.toFloat()

        normalized_bands = ee.Image.cat([normalize_band(band) for band in bands_to_normalize])

        return image_with_indices.addBands(normalized_bands).toFloat()

    def process_month(self, month):
        start_date = dt.date(self.year, month, 1)
        if month == 12:
            end_date = dt.date(self.year + 1, 1, 1)
        else:
            end_date = dt.date(self.year, month + 1, 1)

        sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(self.data_fc) \
            .filterDate(str(start_date), str(end_date)) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
            .map(self.calculate_indices_and_normalize)

        composite = sentinel2.median().toFloat()
        date_str = f'{self.year % 100:02d}{month:02d}'

        def rename_bands(band):
            return ee.String(band).cat('_').cat(date_str)

        bands = composite.bandNames().map(rename_bands)
        composite = composite.rename(bands)

        return composite

    def process_all_months(self):
        composites = [self.process_month(month) for month in self.months]
        combined_image = ee.Image.cat(composites)
        return combined_image

    def add_bands_to_fc(self, image):
        def add_bands(feature):
            sampled = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=10,
                maxPixels=1e13
            )
            return feature.set(sampled)
        return self.data_fc.map(add_bands)

    def export_to_drive(self, description, file_format='GEOJSON'):
        combined_image = self.process_all_months()
        data_with_bands = self.add_bands_to_fc(combined_image)

        export_task = ee.batch.Export.table.toDrive(
            collection=data_with_bands,
            description=description,
            fileFormat=file_format
        )
        export_task.start()

        while export_task.active():
            print('Polling for task (id: {}).'.format(export_task.id))
            time.sleep(30)

        print('Export task completed with status:', export_task.status())

# Usage example
year = 2023
months = [8, 9, 10]  # Example months
indices = ['NDVI', 'EVI', 'SAVI', 'NDWI']  # Example indices to calculate
data_fc= data_2023  # Replace with your data feature collection
processor = Sentinel2Processor(year, months, indices, data_fc)
processor.export_to_drive('data_2023_with_bands_normalized_monthly')


EEException: Collection.loadTable: Collection asset 'projects/ee-kkidia3/assets/vouandzou' not found.

#### b. Plotting some indices

In [ ]:
class SenegalCropMap:
    def __init__(self, crops_data, year, boundaries):
        self.crops_data = crops_data
        self.year = year
        self.boundaries = boundaries
        self.map = None
        self.init_map()

    def calculate_ndvi(self, image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        return image.addBands(ndvi)

    def init_map(self):
        center = self.boundaries.geometry().centroid().getInfo()['coordinates']
        self.map = folium.Map(location=[center[1], center[0]], zoom_start=7, tiles=None)
        folium.TileLayer(
            tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
            attr='Google Satellite',
            name='Google Satellite',
            overlay=False,
            control=True
        ).add_to(self.map)
        folium.TileLayer(
            tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
            attr='OpenStreetMap',
            name='OpenStreetMap',
            overlay=False,
            control=True
        ).add_to(self.map)

    def add_ee_layer(self, ee_image_object, vis_params, name):
        map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
        folium.raster_layers.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
            name=name,
            overlay=True,
            control=True
        ).add_to(self.map)

    def add_crop_layer(self):
        # Generate a unique color for each crop type
        crop_types = set([feature['properties']['Speculatio'] for feature in self.crops_data.getInfo()['features']])
        color_map = {crop: f'#{random.randint(0, 0xFFFFFF):06x}' for crop in crop_types}

        # Define style function using crop types
        def style_function(feature):
            crop = feature['properties'].get('Speculatio', '')
            return {
                'fillColor': color_map.get(crop, '#FFFFFF'),
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.7
            }

        # Add GeoJson layer with pop-ups
        folium.GeoJson(
            self.crops_data.getInfo(),
            style_function=style_function,
            name="Crop Types",
            tooltip=folium.GeoJsonTooltip(fields=['Speculatio'], aliases=['Crop:']),
            popup=folium.GeoJsonPopup(fields=['Speculatio'], aliases=['Crop Type:'], labels=True)
        ).add_to(self.map)

        # Add legend for crop types
        legend_html = '<div style="position: fixed; bottom: 50px; left: 50px; width: 220px; height: 280px; ' \
                      'border:2px solid grey; z-index:9999; font-size:14px; background-color:white; overflow-y: auto;">' \
                      '<div style="padding: 10px;"><strong>Crop Types</strong><br>'
        for crop, color in color_map.items():
            legend_html += f'<span style="background-color: {color}; border: 1px solid black; ' \
                           f'display: inline-block; width: 12px; height: 12px;"></span> {crop}<br>'
        legend_html += '</div></div>'
        self.map.get_root().html.add_child(folium.Element(legend_html))

    def add_layers(self):
        self.add_crop_layer()
        # RGB and NDVI visualization for the specified year
        sentinel = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
            .filterBounds(self.boundaries) \
            .filterDate(f'{self.year}-08-01', f'{self.year}-11-30') \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

        rgb = sentinel.mosaic().clip(self.boundaries) #can change tp mean, median etc
        self.add_ee_layer(rgb, {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}, 'RGB Composite')

        # NDVI and remapped NDVI for each month
        months = ['08', '09', '10', '11']
        month_names = ['August', 'September', 'October', 'November'] #
        ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'red', 'yellow', 'lightgreen', 'darkgreen']}
        ndvi_map_vis = {
            'min': 1,
            'max': 7,
            'palette': ['white', 'blue', 'grey', 'red', 'yellow', 'lightgreen', 'darkgreen']
        }

        for month, month_name in zip(months, month_names):
            start_date = f'{self.year}-{month}-01'
            end_date = f'{self.year}-{month}-30'

            sentinel_month = sentinel.filterDate(start_date, end_date).map(self.calculate_ndvi)
            ndvi_median = sentinel_month.select('NDVI').mosaic().clip(self.boundaries)
            self.add_ee_layer(ndvi_median, ndvi_vis, f'NDVI - {month_name}')

            ndvi_mapped = ee.Image(1) \
                .where(ndvi_median.lte(-0.1), 1) \
                .where(ndvi_median.gt(-0.1).And(ndvi_median.lte(-0.2)), 2) \
                .where(ndvi_median.gt(-0.2).And(ndvi_median.lte(0)), 3) \
                .where(ndvi_median.gt(0).And(ndvi_median.lte(0.1)), 4) \
                .where(ndvi_median.gt(0.1).And(ndvi_median.lte(0.2)), 5) \
                .where(ndvi_median.gt(0.2).And(ndvi_median.lte(0.4)), 6) \
                .where(ndvi_median.gt(0.4), 7)

            self.add_ee_layer(ndvi_mapped, ndvi_map_vis, f'Remapped NDVI - {month_name}')

        folium.LayerControl().add_to(self.map)

    def show_map(self):
        return self.map

#2018
senegal_admin1 = ee.FeatureCollection("projects/ee-janet/assets/senegal/sen_adm1")
map_instance = SenegalCropMap(data_2018, 2018, senegal_admin1) # you can change for other years
map_instance.add_layers()
map_to_show = map_instance.show_map()
map_to_show

# Classification

We will use the exported shapefiles in Dunia for classification

In [ ]:
import geopandas as gpd
data_2018_shp = gpd.read_file('/content/drive/MyDrive/ICRISAT/crop_type_classification/data_2018_with_bands_normalized_monthly.shp')
data_2019_shp = gpd.read_file('/content/drive/MyDrive/ICRISAT/crop_type_classification/data_2019_with_bands_normalized_monthly.geojson')
data_2020_shp = gpd.read_file('/content/drive/MyDrive/ICRISAT/crop_type_classification/data_2020_with_bands_normalized_monthly.geojson') # illdo it again

In [ ]:
data_2018_shp.head()

,GCI_1810,B11_n_1808,B11_n_1809,B12_n_1808,B12_n_1809,B3_1808,B3_1809,Id_1,B5_1810,SAVI_1810,...,B5_1808,B12_n_1810,Sup_ha,NDVI_1810,SAVI_1809,B5_n_1810,SAVI_1808,NDRE_1808,NDRE_1809,geometry
0,1.083424,0.216060,0.244740,0.150677,0.113601,1777.867951,1528.330988,29234,1629.321625,0.596635,...,1989.130886,0.123216,1.701976,0.397809,0.548237,0.232850,0.336413,0.162984,0.263747,"POLYGON ((-16.60330 14.31427, -16.60209 14.314..."
1,0.826673,0.388937,0.409548,0.300421,0.236136,2272.707126,1938.399397,29490,2361.071685,0.321058,...,2832.761366,0.256556,0.379163,0.214059,0.359115,0.358197,0.251664,0.146127,0.187054,"POLYGON ((-16.59843 14.29556, -16.59787 14.295..."
2,1.065002,0.162087,0.210649,0.103234,0.086911,1511.401404,1404.449441,29529,1399.537618,0.584787,...,1664.585655,0.088603,0.184422,0.389915,0.545932,0.195857,0.276683,0.137156,0.259968,"POLYGON ((-16.60472 14.30775, -16.60461 14.307..."
3,1.029934,0.292867,0.302049,0.189620,0.139041,1589.574718,1509.921807,29530,1824.993484,0.417508,...,1921.442224,0.187824,0.094125,0.278370,0.569852,0.272628,0.624998,0.323613,0.289230,"POLYGON ((-16.60452 14.30688, -16.60423 14.306..."
4,1.293020,0.350805,0.343713,0.264489,0.176575,1946.594934,1675.246947,29230,1844.583392,0.560459,...,2437.417086,0.170998,5.083124,0.373679,0.537626,0.272207,0.342953,0.194665,0.280787,"POLYGON ((-16.60636 14.30014, -16.60616 14.300..."


In [ ]:
data_2019_shp.head()

,Admin0,Admin1,Admin2,Admin3,Admin4,Annee,B10_1908,B10_1909,B10_1910,B11_1908,...,SAVI_1908,SAVI_1909,SAVI_1910,Site,SourceD,Speculatio,Sup_ha,Type,id,geometry
0,Senegal,Diourbel,Bambey,Ngoye,Ngoye,2019,21.969206,NaN,13.745123,5029.854328,...,0.160022,NaN,0.607960,,CSE/CIRAD,Millet,1.335229,Cereales,000000000000000026dd,"POLYGON ((-16.37007 14.62108, -16.36996 14.621..."
1,Senegal,Diourbel,Bambey,Ngoye,Ngoye,2019,22.512317,NaN,12.797142,5325.155418,...,0.161407,NaN,0.618715,,CSE/CIRAD,Millet,0.987123,Cereales,000000000000000026de,"POLYGON ((-16.37155 14.62024, -16.37151 14.620..."
2,Senegal,Diourbel,Bambey,Ngoye,Ngoye,2019,23.714598,NaN,13.258093,4881.975941,...,0.162699,NaN,0.792638,,CSE/CIRAD,Millet,0.326304,Cereales,000000000000000026e0,"POLYGON ((-16.41202 14.61126, -16.41202 14.610..."
3,Senegal,Diourbel,Bambey,Ngoye,Ngoye,2019,23.028137,NaN,12.975198,4352.526469,...,0.157794,NaN,0.784191,,CSE/CIRAD,Millet,0.098086,Cereales,000000000000000026e2,"POLYGON ((-16.41165 14.61155, -16.41160 14.611..."
4,Senegal,Diourbel,Bambey,Ngoye,Ngoye,2019,22.500000,NaN,12.500000,4256.681262,...,0.151105,NaN,0.658311,,CSE/CIRAD,Millet,0.042732,Cereales,000000000000000026e3,"POLYGON ((-16.41158 14.61142, -16.41147 14.611..."


In [ ]:
data_2020_shp.head(30)

,Admin0,Admin1,Admin2,Admin3,Admin4,Annee,B10_2008,B10_2009,B10_2010,B11_2008,...,SAVI_2008,SAVI_2009,SAVI_2010,Site,SourceD,Speculatio,Sup_ha,Type,id,geometry
0,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Mixte (mil nieb,0.507473,,00000000000000006283,"POLYGON ((-13.18952 15.49683, -13.18950 15.496..."
1,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Mixte (mil nieb,0.993971,,00000000000000006289,"POLYGON ((-13.18880 15.49704, -13.18875 15.497..."
2,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Mixte (mil biss,0.325075,,0000000000000000628a,"POLYGON ((-13.19030 15.49880, -13.19029 15.498..."
3,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Groundnut_mixed,0.557906,,0000000000000000628b,"POLYGON ((-13.18960 15.49871, -13.18959 15.498..."
4,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Mixte (mil nieb,0.740876,,0000000000000000628e,"POLYGON ((-13.19184 15.49865, -13.19175 15.498..."
5,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Mixte (mil nieb,2.783540,,0000000000000000628f,"POLYGON ((-13.19740 15.49527, -13.19724 15.495..."
6,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Mixte (mil nieb,2.243038,,00000000000000006290,"POLYGON ((-13.19740 15.49527, -13.19739 15.495..."
7,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Mixte (mil nieb,1.610484,,00000000000000006293,"POLYGON ((-13.19601 15.49521, -13.19601 15.495..."
8,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Mixte (mil nieb,2.145073,,00000000000000006295,"POLYGON ((-13.19874 15.49450, -13.19871 15.494..."
9,Senegal,Matam,Kanel,Kanel,Kanel,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,CSE/SERVIR,Mixte (mil nieb,3.062775,,00000000000000006299,"POLYGON ((-13.19313 15.49520, -13.19296 15.495..."


In [ ]:
data_2028_shp.columns[:30]

Index(['GCI_1810', 'B11_n_1808', 'B11_n_1809', 'B12_n_1808', 'B12_n_1809',
       'B3_1808', 'B3_1809', 'Id_1', 'B5_1810', 'SAVI_1810', 'GCI_1808',
       'NDBI_1808', 'B4_1810', 'GCI_1809', 'NDBI_1809', 'id', 'B8A_n_1808',
       'B8A_n_1809', 'B8A_n_1810', 'Crop_Ncrop', 'B11_n_1810', 'Speculatio',
       'NBR_1810', 'B4_1809', 'B4_1808', 'B1_1808', 'Site', 'EVI_1810',
       'QA60_1810', 'B6_1808'],
      dtype='object')

In [ ]:
from google.colab import files
uploaded = files.upload()
for file in os.listdir("/content"): print(file)

In [ ]:
import os
import geopandas as gpd
# List files in the /content directory
for file in os.listdir("/content"):
    print(file)


In [ ]:
# Load the Shapefile
data_2018_gdf = gpd.read_file("")

# Verify the data has been loaded
print(data_2018_gdf.head())

### Evaluating the separability of crops

1.   Evaluate the plot of NDVI  for each crop in each region
2.   Compare the NDVI for each crop in all regions